# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from dotenv import load_dotenv
import os
import json

# Import the Geoapify API key
load_dotenv()
api_key = os.getenv('GEOAPIFY_KEY')

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv", keep_default_na=False)

# Rename some columns to make the units of each column more clear
city_data_df = city_data_df.rename(columns={'Max Temp': 'Max Temp (C)', 'Humidity': 'Humidity (%)', 'Cloudiness': 'Cloudiness (%)', 'Wind Speed': 'Wind Speed (m/s)'})

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp (C),Humidity (%),Cloudiness (%),Wind Speed (m/s),Country,Date
0,0,walvis bay,-22.9575,14.5053,13.38,84,3,1.26,NA,1722721405
1,1,olonkinbyen,70.9221,-8.7187,5.49,91,90,3.08,SJ,1722721576
2,2,bethel,41.3712,-73.4140,26.47,91,75,1.54,US,1722721342
3,3,saint-francois,46.4154,3.9054,18.69,83,0,2.57,FR,1722721578
4,4,grytviken,-54.2811,-36.5092,-4.82,91,100,4.10,GS,1722721580


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points('Lng', 'Lat', geo=True, tiles= 'OSM', frame_width= 700, frame_height= 500, size= 'Humidity (%)', scale=0.8, color= 'City', alpha=0.6)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity (%))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_city_data = city_data_df.loc[(city_data_df['Max Temp (C)'] < 24) & (city_data_df['Max Temp (C)'] > 15) & (city_data_df['Cloudiness (%)'] == 0) & (city_data_df['Wind Speed (m/s)'] < 3), :]


# Drop any rows with null values
narrow_city_data = narrow_city_data.dropna(how='any')

# Display sample data
narrow_city_data

,City_ID,City,Lat,Lng,Max Temp (C),Humidity (%),Cloudiness (%),Wind Speed (m/s),Country,Date
3,3,saint-francois,46.4154,3.9054,18.69,83,0,2.57,FR,1722721578
83,83,tromso,69.6496,18.9570,16.01,67,0,0.51,NO,1722721697
133,133,zhangatas,43.5667,69.7500,23.56,25,0,2.24,KZ,1722721760
147,147,targuist,34.9377,-4.3186,22.54,30,0,1.48,MA,1722721777
230,230,lichinga,-13.3128,35.2406,15.05,67,0,2.24,MZ,1722721879
234,234,giyani,-23.3025,30.7187,18.90,45,0,0.63,ZA,1722721884
257,257,ghanzi,-21.5667,21.7833,17.09,25,0,2.87,BW,1722721910
310,310,tsiombe,-25.3000,45.4833,16.76,87,0,0.75,MG,1722721973
394,394,lata,40.1629,-8.3327,18.37,92,0,1.15,PT,1722722074
477,477,soio,-6.1349,12.3689,19.99,89,0,2.94,AO,1722722173


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_city_data[['City', 'Country', 'Lat', 'Lng', 'Humidity (%)']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity (%),Hotel Name
3,saint-francois,FR,46.4154,3.9054,83,
83,tromso,NO,69.6496,18.9570,67,
133,zhangatas,KZ,43.5667,69.7500,25,
147,targuist,MA,34.9377,-4.3186,30,
230,lichinga,MZ,-13.3128,35.2406,67,
234,giyani,ZA,-23.3025,30.7187,45,
257,ghanzi,BW,-21.5667,21.7833,25,
310,tsiombe,MG,-25.3000,45.4833,87,
394,lata,PT,40.1629,-8.3327,92,
477,soio,AO,-6.1349,12.3689,89,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
limit = 10
categories = 'accommodation.hotel'


params = {
    "categories": categories,
    "limit": limit,
    "apiKey": api_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saint-francois - nearest hotel: Chez Lily
tromso - nearest hotel: Clarion Collection Hotel Aurora
zhangatas - nearest hotel: Арман
targuist - nearest hotel: No hotel found
lichinga - nearest hotel: Residencial 2+1
giyani - nearest hotel: No hotel found
ghanzi - nearest hotel: Ghanzi farmhouse
tsiombe - nearest hotel: No hotel found
lata - nearest hotel: Residencial Botânico
soio - nearest hotel: Hotel Maria Teres
opuwo - nearest hotel: Ohakane Lodge
karkira - nearest hotel: استراحة دريم هاوس
macia - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity (%),Hotel Name
3,saint-francois,FR,46.4154,3.9054,83,Chez Lily
83,tromso,NO,69.6496,18.9570,67,Clarion Collection Hotel Aurora
133,zhangatas,KZ,43.5667,69.7500,25,Арман
147,targuist,MA,34.9377,-4.3186,30,No hotel found
230,lichinga,MZ,-13.3128,35.2406,67,Residencial 2+1
234,giyani,ZA,-23.3025,30.7187,45,No hotel found
257,ghanzi,BW,-21.5667,21.7833,25,Ghanzi farmhouse
310,tsiombe,MG,-25.3000,45.4833,87,No hotel found
394,lata,PT,40.1629,-8.3327,92,Residencial Botânico
477,soio,AO,-6.1349,12.3689,89,Hotel Maria Teres


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points('Lng', 'Lat', geo=True, tiles= 'OSM', frame_width= 700, frame_height= 500, size= 'Humidity (%)', color= 'City', alpha=0.6, hover_cols=['Country', 'Hotel Name'])

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity (%),Country,Hotel Name)